In [1]:
from apex import *
from proofs import *
from perf_data import *

ZigZag nodes: 33554432


In [2]:
z = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=64*GiB, merkle_hash=pedersen)

In [3]:
(l1, savings, savings_, best_) = optimize(z); (l, savings, savings_, best_)

(5, 11999.659715109738, '03:19:59.7', '180:56:28.9')

In [4]:
a = apex(z, l); a

ZigZag(instance=None, circuit_proof_size=192, hash_size=32, size=68719476736, partitions=8, merkle_hash=HashFunction(hash_time=1.7993e-05, constraints=1152), kdf_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), security=Security(base_degree=5, expansion_degree=8, layers=10, total_challenges=8848, sloth_iter=0), merkle_pessimization=1.5, processor_power=165, relax_time=1, constraint_proving_time=5.013000000000001e-05, apex_height=5)

In [5]:
zz = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=64*GiB, merkle_hash=blake2s)

In [6]:
(ll, savings, savings_, best_) = optimize(zz); (ll, savings, savings_, best_)

(25, 62929.66972253361, '17:28:49.7', '10:35:28.3')

In [7]:
aa = apex(zz, ll); aa

ZigZag(instance=None, circuit_proof_size=192, hash_size=32, size=68719476736, partitions=8, merkle_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), kdf_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), security=Security(base_degree=5, expansion_degree=8, layers=10, total_challenges=8848, sloth_iter=0), merkle_pessimization=1.5, processor_power=165, relax_time=1, constraint_proving_time=5.013000000000001e-05, apex_height=25)

In [8]:
z.show_times()

groth: 02:49:18.2; replication: 181:27:10.4; seal: 184:16:28.5


In [9]:
a.show_times()

groth: 02:25:54.4; replication: 181:27:10.4; seal: 183:53:4.8


In [10]:
a.groth_proving_time()/z.groth_proving_time()

0.8618104115618622

In [11]:
zz.show_times()

groth: 22:41:27.2; replication: 05:22:50.8; seal: 28:04:18.0


In [12]:
aa.show_times()

groth: 05:13:6.4; replication: 05:22:21.9; seal: 10:35:28.3


In [13]:
aa.groth_proving_time()/zz.groth_proving_time()

0.22997946611909653

In [14]:
zz.constraints()

5560733528.0

In [15]:
aa.constraints()

1278854528.0

In [16]:
zz.constraints() - aa.constraints()

4281879000.0

In [17]:
aa.constraints() / zz.constraints()

0.2299794661190965

In [18]:
projected = ZigZag(security=proofs.filecoin_security_requirements, instance=projected_instance, partitions=8)

In [19]:
optimize(projected)

(5, 9.026515571866184, '00:00:9.0', '166:11:47.7')

In [20]:
apex_projected = apex(projected, 13)

In [21]:
apex_projected.show_times(64*GiB)

groth: 12:30:5.6; replication: 153:55:36.5; seal: 166:26:13.9


In [22]:
apex_projected.meets_performance_requirements(64*GiB, filecoin_scaling_requirements)

False

In [23]:
projected_blake = projected.scaled_for_new_hash(blake2s)

In [24]:
(lx, _, _, _) = optimize(projected_blake); lx

5

In [25]:
apex_projected_blake = apex(projected_blake, lx)

In [26]:
apex_projected_blake.show_times(64 * GiB)

groth: 88:53:33.4; replication: 36:33:35.3; seal: 125:27:40.6


In [27]:
projected_blake.show_times(64 * GiB)

groth: 88:54:52.6; replication: 36:33:35.3; seal: 125:28:59.7


In [28]:
projected_blake.meets_performance_requirements(64*GiB, filecoin_scaling_requirements)

True

In [29]:
(ly, _, savings, best) = optimize(projected_blake); (l, savings, best)

(5, '00:01:19.1', '125:27:40.6')

In [30]:
apex_projected_blake = apex(projected_blake, ly)

In [31]:
apex_projected_blake.meets_performance_requirements(64*GiB, filecoin_scaling_requirements)

True

In [32]:
apex_projected_blake.constraints() / apex_projected_blake.partitions # how many constraints in each partition

798131514.0125556

In [33]:
ZigZag(partitions=16).proof_size() <= filecoin_scaling_requirements.proof_size * 128 * GiB

True

Let's work through this again with 128 GiB sectors:

In [34]:
simulated128 = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=128*GiB, merkle_hash=blake2s)

In [35]:
projected.constraints(), projected.constraints(), simulated128.constraints()

(879643632, 879643632, 5732008688.0)

In [36]:
optimize(projected)

(5, 9.026515571866184, '00:00:9.0', '166:11:47.7')

In [37]:
humanize_bytes(apex_projected.groth_proving_memory())

'1.2 TB'

In [38]:
apex_projected.meets_performance_requirements(128*GiB, filecoin_scaling_requirements)

False

In [39]:
apex_projected_blake.meets_performance_requirements(128*GiB, filecoin_scaling_requirements)

True

In [40]:
apex_projected_blake.meets_performance_requirements(64*GiB, filecoin_scaling_requirements)

True

Still good.

In [41]:
humanize_bytes(apex_projected.minimum_viable_sector_size(filecoin_scaling_requirements))

'0.0 bytes'

In [42]:
humanize_bytes(apex_projected_blake.minimum_viable_sector_size(filecoin_scaling_requirements))

'64.0 GB'

In [43]:
apex_projected.constraints(),  apex_projected_blake.constraints()

(881910377.4222336, 6385052112.100445)

In [44]:
apex_projected.total_seal_time() / apex_projected_blake.total_seal_time()

1.326602269489586

In [45]:
apex_projected.show_times()

groth: 12:30:5.6; replication: 153:55:36.5; seal: 166:26:13.9


In [46]:
apex_projected_blake.show_times()

groth: 88:53:33.4; replication: 36:33:35.3; seal: 125:27:40.6


In [47]:
apex_projected.apex_constraints_avoided()/ apex_projected.apex_constraints()

0.0389934276923077

In [48]:
apex_projected_blake.apex_constraints_avoided() / apex_projected_blake.apex_constraints()

3.5484019200000008

In [49]:
apex_projected.total_seal_time() / projected.total_seal_time()

1.0014326484954814

In [50]:
apex_projected.groth_proving_time(), projected.groth_proving_time()

(45005.59085629257, 44096.53527216001)

In [51]:
projected.show_times()

groth: 12:14:56.5; replication: 153:56:28.4; seal: 166:11:56.7


In [52]:
aa.apex_constraints_avoided()

1648516.5639936004

In [53]:
(aa.apex_height - 1) * (aa.degree() + 2) * aa.total_challenges() * aa.merkle_hash.constraints **aa.constraint_proving_time

3186756.122978253

In [54]:
aa.constraints()

1278854528.0

### NOTE

Here is the point at which I realize that we need a distinct apex (and the cost of commiting to it) per layer. Then also, that because parent selection is pseudorandom, we cannot use the trick of bucketing challenges evenly within apex leaves except for the data and replica nodes. Sadness descends on the late night.